<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

import matplotlib.pyplot as plt


from libvis import Vis

No module named 'bokeh'


In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [3]:
vis = Vis(ws_port=7701, vis_port=7001)
vis.start()

Starting http at 7001
2020-05-17 23:02:16.091 | INFO     | legimens.websocket.server:start_server:33 - Starting ws server at localhost:7701


In [4]:
losses = [1]
vis.watch(losses, 'loss')

'Legi_0x7fcf8ca6cae0'

In [25]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [44]:
use_cuda = False

device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

epochs=1
batch_size=2000
model = Net().to(device)

In [45]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)
    
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [48]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303627
Train Epoch: 1 [2000/60000 (3%)]	Loss: 5.380020
Train Epoch: 1 [4000/60000 (7%)]	Loss: 14.021826
Train Epoch: 1 [6000/60000 (10%)]	Loss: 18.893517
Train Epoch: 1 [8000/60000 (13%)]	Loss: 7.992403
Train Epoch: 1 [10000/60000 (17%)]	Loss: 8.522062
Train Epoch: 1 [12000/60000 (20%)]	Loss: 11.277676
Train Epoch: 1 [14000/60000 (23%)]	Loss: 4.811124
Train Epoch: 1 [16000/60000 (27%)]	Loss: 4.622343
Train Epoch: 1 [18000/60000 (30%)]	Loss: 2.006118
Train Epoch: 1 [20000/60000 (33%)]	Loss: 1.475107
Train Epoch: 1 [22000/60000 (37%)]	Loss: 1.607400
Train Epoch: 1 [24000/60000 (40%)]	Loss: 1.497755
Train Epoch: 1 [26000/60000 (43%)]	Loss: 1.139499
Train Epoch: 1 [28000/60000 (47%)]	Loss: 1.168181
Train Epoch: 1 [30000/60000 (50%)]	Loss: 1.098559
2020-05-17 23:25:47.905 | INFO     | legimens.App:_start:197 - App crashed.


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/site-packages/trio/_core/_run.py", line 1804, in run
    raise runner.main_task_outcome.error
  File "/home/dali/side-projects-hobby/legimens/python/legimens/App.py", line 194, in _start
    nursery.start_soon(self._poll_objects)
  File "/usr/lib/python3.8/site-packages/trio/_core/_run.py", line 730, in __aexit__
    raise combined_error_from_nursery
  File "/home/dali/side-projects-hobby/legimens/python/legimens/App.py", line 150, in _poll_objects
    for ref_ in self._watched_children:
  File "/usr/lib/python3.8/weakref.py", line 217, in keys
    for k, wr in self.data.items():
RuntimeError: dictionary changed size during iteration


Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.138399
Train Epoch: 1 [34000/60000 (57%)]	Loss: 1.070479
Train Epoch: 1 [36000/60000 (60%)]	Loss: 0.982675
Train Epoch: 1 [38000/60000 (63%)]	Loss: 0.942731
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.868461
Train Epoch: 1 [42000/60000 (70%)]	Loss: 0.814603
Train Epoch: 1 [44000/60000 (73%)]	Loss: 0.813016
Train Epoch: 1 [46000/60000 (77%)]	Loss: 0.764307
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.695083
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.678800
Train Epoch: 1 [52000/60000 (87%)]	Loss: 0.761719
Train Epoch: 1 [54000/60000 (90%)]	Loss: 0.637173
Train Epoch: 1 [56000/60000 (93%)]	Loss: 0.667314
Train Epoch: 1 [58000/60000 (97%)]	Loss: 0.587480

Test set: Average loss: 0.3503, Accuracy: 9074/10000 (91%)



In [46]:
from libvis import interface as ifc

def ser(x):
    return x.clone().cpu().detach().numpy().tolist()

ser(model.fc2.weight)

[[0.0747339054942131,
  0.06502559036016464,
  -0.011352866888046265,
  -0.03932381048798561,
  0.033889755606651306,
  0.00952131301164627,
  0.01662427932024002,
  0.047617651522159576,
  -0.06891325116157532,
  0.02298174798488617,
  0.005075432360172272,
  0.029170885682106018,
  0.01320202648639679,
  -0.06365430355072021,
  0.00433821976184845,
  0.030349060893058777,
  0.04055101424455643,
  0.06960802525281906,
  0.03253711014986038,
  0.05121753364801407,
  0.07244560867547989,
  -0.07187788933515549,
  -0.026776481419801712,
  -0.04387608915567398,
  -0.06089285761117935,
  0.04847913235425949,
  0.013186030089855194,
  0.05350945144891739,
  0.012946166098117828,
  -0.08772548288106918,
  0.019848033785820007,
  -0.028702963143587112,
  -0.03698338568210602,
  0.04043949395418167,
  0.05571960657835007,
  -0.021685555577278137,
  -0.04392228275537491,
  0.001185685396194458,
  0.06683220714330673,
  0.07275024801492691,
  -0.07609938830137253,
  0.004530787467956543,
  0.084

In [47]:
vis.watch(model.fc2.weight, 'conv1', serializer=ser)
vis.vars.conv1 = vis.vars['conv1']

2020-05-17 23:24:21.308 | ERROR    | legimens.App:_handler:91 - Handling <class 'trio_websocket._impl.ConnectionClosed'> error:CloseReason<code=1000, name=NORMAL_CLOSURE, reason=None>
2020-05-17 23:24:21.309 | INFO     | legimens.App:_handler:76 - New ws connection of /Legi_0x7fcf77fa3360 from ::1
2020-05-17 23:24:21.315 | INFO     | legimens.App:_handle_obj_ref:107 - Yield initial update {"body": {"value": [[0.0747339054942131, 0.06502559036016464, -0.011352866888046265, -0.03932381048798561, 0.033889755606651306, 0.00952131301164627, 0.01662427932024002, 0.047617651522159576, -0.06891325116157532, 0.02298174798488617, 0.005075432360172272, 0.029170885682106018, 0.01320202648639679, -0.06365430355072021, 0.00433821976184845, 0.030349060893058777, 0.04055101424455643, 0.06960802525281906, 0.03253711014986038, 0.05121753364801407, 0.07244560867547989, -0.07187788933515549, -0.026776481419801712, -0.04387608915567398, -0.06089285761117935, 0.04847913235425949, 0.013186030089855194, 0.053

In [40]:
model.fc2.weight.shape)

torch.nn.parameter.Parameter

In [15]:
ifc.serialize_to_vis(vis.vars.conv1.body)

{'value': [[-0.19980235397815704, 0.26904115080833435, 0.18667030334472656],
  [0.26798078417778015, 0.0039983587339520454, 0.23471753299236298],
  [0.05251083895564079, 0.12333120405673981, 0.13972367346286774]],
 'type': 'Parameter'}

In [35]:
print()
vis.app._watch_poll_delay = .5